In [30]:
import pandas as pd
import os
import glob
import pandas as pd

def transform_data(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path)
    
    # Verificar que las columnas necesarias estén presentes
    required_columns = ['year', 'month', 'cluster', 'depth_m', 'thetao']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Crear un nuevo DataFrame con las columnas year, month y cluster
    new_df = df[['year', 'month', 'cluster']].drop_duplicates().reset_index(drop=True)
    
    # Iterar sobre cada combinación única de year, month y cluster
    for (year, month, cluster), group in df.groupby(['year', 'month', 'cluster']):
        # Crear un DataFrame temporal para almacenar los valores de thetao por profundidad
        temp_df = group[['depth_m', 'thetao']]
        
        # Iterar sobre valores únicos de depth_m
        for depth in temp_df['depth_m'].unique():
            # Tratar de convertir depth a float y formatear, si no se puede, usar como string
            try:
                depth_formatted = f'{float(depth):.2f}'
            except ValueError:
                depth_formatted = depth
            
            # Filtrar los valores de thetao para la profundidad actual
            value = temp_df[temp_df['depth_m'] == depth]['thetao'].values[0]
            
            # Asignar el valor al nuevo DataFrame, manejando los índices correctamente
            new_df.loc[(new_df['year'] == year) & (new_df['month'] == month) & (new_df['cluster'] == cluster), f'depth_{depth_formatted}'] = value
    
    return new_df


def process_folder(input_folder, output_folder):
    # Crear el directorio de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    # Listar todos los archivos CSV en la carpeta de entrada
    csv_files = glob.glob(os.path.join(input_folder, '*.csv'))
    
    for file_path in csv_files:
        # Transformar los datos
        transformed_df = transform_data(file_path)
        
        # Obtener el nombre del archivo sin la carpeta
        file_name = os.path.basename(file_path)
        
        # Crear la ruta completa para el archivo de salida
        output_file_path = os.path.join(output_folder, f'transformed_{file_name}')
        
        # Guardar el DataFrame transformado en un nuevo archivo CSV
        transformed_df.to_csv(output_file_path, index=False)
        
        print(f'Archivo transformado guardado en: {output_file_path}')


In [31]:

# Usar la función para procesar una carpeta
input_folder = 'data/futuretemp'  # Cambia esto por la ruta de tu carpeta de entrada
output_folder = 'data/forecast_data'  # Cambia esto por la ruta de tu carpeta de salida
process_folder(input_folder, output_folder)


Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2073_clust7_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2039_clust3_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2062_clust5_r1i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2055_clust4_r1i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2079_clust7_r1i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2025_clust0_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2055_clust0_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2068_clust2_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/transformed_cluster_temp_yr2068_clust3_r2i1p1f1.csv
Archivo transformado guardado en: data/forecast_data/tr

In [34]:
# Especifica la carpeta que contiene los archivos
folder_path = 'data/forecast_data'

# Lista para almacenar los DataFrames
dataframes = []

# Listar todos los archivos en la carpeta
for file_name in os.listdir(folder_path):
    # Crear la ruta completa del archivo
    file_path = os.path.join(folder_path, file_name)
    # Verificar si el archivo es un archivo CSV
    if file_name.endswith('.csv'):
        # Leer el archivo CSV y agregarlo a la lista de DataFrames
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
merged_df = pd.concat(dataframes, ignore_index=True)
# Mover las columnas 'depth_mean_10m' y 'depth_mean_30m' después de 'cluster'
columns = list(merged_df.columns)
cluster_index = columns.index('cluster')
columns.remove('depth_mean_10m')
columns.remove('depth_mean_30m')
columns.insert(cluster_index + 1, 'depth_mean_10m')
columns.insert(cluster_index + 2, 'depth_mean_30m')
merge_df = merged_df[columns]
# Renombrar las columnas especificadas
rename_mapping = {
    'cluster': 'Cluster_Label',
    'depth_mean_10m': 'mean_temp_10m',
    'depth_mean_30m': 'mean_temp_30m',
    'depth_6.00': 'thetao_sfc=6',
    'depth_7.93': 'thetao_sfc=7.92956018447876',
    'depth_9.57': 'thetao_sfc=9.572997093200684',
    'depth_11.40': 'thetao_sfc=11.40499973297119',
    'depth_13.47': 'thetao_sfc=13.46714019775391',
    'depth_15.81': 'thetao_sfc=15.8100700378418',
    'depth_18.50': 'thetao_sfc=18.49555969238281',
    'depth_21.60': 'thetao_sfc=21.59881973266602',
    'depth_25.21': 'thetao_sfc=25.21141052246094',
    'depth_29.44': 'thetao_sfc=29.44473075866699'
}
merge_df.rename(columns=rename_mapping, inplace=True)
# Guardar el DataFrame resultante en un archivo CSV llamado data.csv
merged_df.to_csv('future_data.csv', index=False)

print("Archivos combinados y guardados en 'data.csv'.")

Archivos combinados y guardados en 'data.csv'.


In [60]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Directorio donde se almacenan los modelos y los scalers
models_directory = 'modelos_moe'  # Cambia esto a la ruta real de tu directorio de modelos

# Función para cargar el modelo y los scalers
def load_model_and_scalers(species_name, cluster_label):
    model_filename = os.path.join(models_directory, f'{species_name}_cluster_{cluster_label}_moe_model.h5')
    scaler_filename = os.path.join(models_directory, f'{species_name}_cluster_{cluster_label}_scaler.pkl')

    # Cargar el modelo (usando TensorFlow/Keras para el archivo .h5)
    model = tf.keras.models.load_model(model_filename)
    
    # Cargar el scaler (usando joblib para el archivo .pkl)
    scaler = joblib.load(scaler_filename)

    return model, scaler

def load_historical_data(species_name, cluster_label):
    # Ruta al archivo CSV con los datos históricos
    historical_data_path = os.path.join(models_directory, f'{species_name}_cluster_{cluster_label}_historical_data.csv')
    historical_data = pd.read_csv(historical_data_path)
    historical_data['date'] = pd.to_datetime(historical_data['date'])
    return historical_data

def make_forecast_with_historical_data(model, scaler, historical_data, new_data, look_back=6):
    features = ['Cluster_Label', 'mean_temp_30m', 'mean_temp_10m', 'thetao_sfc=6',
                'thetao_sfc=7.92956018447876', 'thetao_sfc=9.572997093200684', 'thetao_sfc=11.40499973297119',
                'thetao_sfc=13.46714019775391', 'thetao_sfc=15.8100700378418', 'thetao_sfc=18.49555969238281',
                'thetao_sfc=21.59881973266602', 'thetao_sfc=25.21141052246094', 'thetao_sfc=29.44473075866699']
    
    # Obtener los últimos 'look_back' valores históricos
    historical_subset = historical_data[-look_back:]
    
    # Escalar los datos históricos
    historical_scaled = scaler.transform(historical_subset[features])
    
    # Preparar los nuevos datos con el formato adecuado
    new_data_scaled = scaler.transform(new_data[features])
    
    # Preparar la entrada para el modelo
    X = np.vstack([historical_scaled, new_data_scaled[:1]])
    X = np.reshape(X, (1, X.shape[0], X.shape[1]))
    
    # Hacer el pronóstico
    prediction_scaled = model.predict(X)
    
    # Desescalar la predicción
    temp = np.zeros((prediction_scaled.shape[0], len(scaler.feature_names_in_)))
    temp[:, -1] = prediction_scaled[:, 0]
    prediction = scaler.inverse_transform(temp)[:, -1][0]
    
    return prediction

def update_data_with_prediction(historical_data, new_data, prediction):
    # Agregar el pronóstico al conjunto de datos
    new_row = new_data.iloc[0].copy()
    new_row['landed_w_kg'] = prediction
    updated_data = historical_data.append(new_row, ignore_index=True)
    return updated_data

def iterative_forecast(species_name, cluster_label, new_data, look_back=6):
    model, scaler = load_model_and_scalers(species_name, cluster_label)
    historical_data = load_historical_data(species_name, cluster_label)
    
    predictions = []
    for i in range(len(new_data)):
        prediction = make_forecast_with_historical_data(model, scaler, historical_data, new_data.iloc[[i]], look_back)
        predictions.append(prediction)
        historical_data = update_data_with_prediction(historical_data, new_data.iloc[[i]], prediction)
    
    return predictions

In [ ]:
# Ejemplo de uso
species_name = 'CABRILLA'  # Reemplaza con el nombre real de la especie
cluster_label = 1  # Reemplaza con el clúster real

In [ ]:
# Hacer el pronóstico iterativo
predictions = iterative_forecast(species_name, cluster_label, new_data)
print(predictions)

In [59]:
# Ejemplo de uso
species_name = 'CABRILLA'  # Reemplaza con el nombre real de la especie
cluster_label = 1  # Reemplaza con el clúster real
# Hacer el pronóstico
new_data=merge_df[merge_df['Cluster_Label']==1]
predictions = make_forecast(species_name, cluster_label, new_data)
print(predictions)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- landed_w_kg


In [50]:
print(f'{species_name}_cluster_{format_cluster_label}_moe_model.h5')

NameError: name 'format_cluster_label' is not defined

In [53]:
model = joblib.load('modelos_moe/CABRILLA_cluster_1_moe_model.h5' )
scaler = joblib.load('modelos_moe/CABRILLA_cluster_1_moe_scaler.pkl')

KeyError: 72